## Laboratorio 6 - Avances

### 1) Descargar los archivos de datos (traficogt.txt)

Para el laboratorio se decidió utilizar el archivo de tráficogt.txt ya que el tema parece bastante interesante y tiene mucho potencial para ser analizado.

Importaciones

In [11]:
import pandas as pd
import json
import re
import string
import unidecode
import nltk
from nltk.corpus import stopwords
import networkx as nx
import chardet

### 2) Cargar los archivos a Python

In [12]:
# Encontrar la codificación del archivo
with open("traficogt.txt", "rb") as f:
    rawdata = f.read(100000)  # leer un bloque
    print(chardet.detect(rawdata))

{'encoding': 'UTF-16', 'confidence': 1.0, 'language': ''}


Vemos que la codificación detectada es UTF-16 y lo aplicamos para poder cargar el archivo a python.

In [14]:
# Cargar tweets desde archivo JSONL (cada línea es un JSON)
tweets = []
with open("traficogt.txt", "r", encoding="utf-16") as f:
    for line_num, line in enumerate(f):
        try:
            tweets.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Skipping line {line_num + 1} due to JSONDecodeError: {e}")
        except Exception as e:
            print(f"Skipping line {line_num + 1} due to unexpected error: {e}")


df = pd.json_normalize(tweets)  # Convertir a DataFrame
display(df.head(3))

Skipping line 5605 due to JSONDecodeError: Expecting ',' delimiter: line 1 column 4314 (char 4313)


,id,id_str,url,date,lang,rawContent,replyCount,retweetCount,likeCount,quoteCount,...,quotedTweet.quotedTweet.inReplyToTweetIdStr,quotedTweet.quotedTweet.inReplyToUser,quotedTweet.quotedTweet.source,quotedTweet.quotedTweet.sourceUrl,quotedTweet.quotedTweet.sourceLabel,quotedTweet.quotedTweet.media.photos,quotedTweet.quotedTweet.media.videos,quotedTweet.quotedTweet.media.animated,quotedTweet.quotedTweet.card,quotedTweet.quotedTweet._type
0,1834236045598056867,1834236045598056867,https://x.com/traficogt/status/183423604559805...,2024-09-12 14:22:06+00:00,es,Es comprensible la resolución... El ruso sabe ...,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1834029142565658846,1834029142565658846,https://x.com/monymmorales/status/183402914256...,2024-09-12 00:39:56+00:00,es,La corrupción de la @CC_Guatemala\nes descarad...,0,56,84,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1834039491826180424,1834039491826180424,https://x.com/animaldgalaccia/status/183403949...,2024-09-12 01:21:04+00:00,qme,@PNCdeGuatemala @mingobguate @FJimenezmingob @...,0,0,1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se omitió la línea 5605 porque el tweet estaba dañado o no cumplia con el formato, entonces se descartó. Esto es normal y no afecta el análisis.

Se cargaron 5604 tweets

Al expandir el JSON se obtuvo un dataframe con 210 columnas, que incluyen metadatos como id, date, user.username, rawContent, etc. Estos son los datos a los que se les realizará la limpieza.

### 3.1) Limpieza y preprocesamiento de los datos

In [17]:
nltk.download("stopwords")
spanish_stopwords = set(stopwords.words("spanish"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    text = re.sub(r"[0-9]+", "", text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = "".join(c for c in text if c.isalnum() or c.isspace())
    text = unidecode.unidecode(text)
    tokens = [t for t in text.split() if t not in spanish_stopwords]
    return " ".join(tokens)

df["clean_text"] = df["rawContent"].astype(str).apply(clean_text)
df[["rawContent","clean_text"]].head(5)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,rawContent,clean_text
0,Es comprensible la resolución... El ruso sabe ...,comprensible resolucion ruso sabe engrasar maq...
1,La corrupción de la @CC_Guatemala\nes descarad...,corrupcion descarada falsificacion documentos ...
2,@PNCdeGuatemala @mingobguate @FJimenezmingob @...,
3,@amilcarmontejo @AztecaNoticiaGT @BancadaSemil...,
4,@soy_502 @AztecaNoticiaGT @CONAPgt @DenunciaEM...,urgente zona deterioro tala inmoderada trafico...


### 3.2) Extracción de metadatos para identificar las relaciones entre usuarios

In [19]:
edges = []

for _, row in df.iterrows():
    user = row["user.username"]

    # Menciones
    if isinstance(row.get("mentionedUsers"), list):
        for mention in row["mentionedUsers"]:
            edges.append((user, mention["username"], "mention"))

    # Retweets
    if row.get("retweetedTweet") is not None and isinstance(row["retweetedTweet"], dict):
        rt_user = row["retweetedTweet"]["user"]["username"]
        edges.append((user, rt_user, "retweet"))

    # Respuestas
    if row.get("inReplyToUser") is not None and isinstance(row["inReplyToUser"], dict):
        reply_user = row["inReplyToUser"]["username"]
        edges.append((user, reply_user, "reply"))

edges_df = pd.DataFrame(edges, columns=["source", "target", "type"])
display(edges_df.head(10))

,source,target,type
0,monymmorales,CC_Guatemala,mention
1,animaldgalaccia,PNCdeGuatemala,mention
2,animaldgalaccia,mingobguate,mention
3,animaldgalaccia,FJimenezmingob,mention
4,animaldgalaccia,diegoedeleon,mention
5,animaldgalaccia,amilcarmontejo,mention
6,animaldgalaccia,traficogt,mention
7,EstacionDobleA,amilcarmontejo,mention
8,EstacionDobleA,AztecaNoticiaGT,mention
9,EstacionDobleA,BancadaSemilla,mention


Aquí se ve una tabla donde source hace referencia al usuario que emitió el tweet. target contiene el usuario al que va dirigida la interacción y type es el tipo de interacción que puede ser mention, retweet o reply.

### 3.3) Eliminación de duplicados y normalización de nombres de usuario y menciones

In [20]:
edges_df.drop_duplicates(inplace=True)
edges_df["source"] = edges_df["source"].str.lower()
edges_df["target"] = edges_df["target"].str.lower()

### 3.4) Creación de estructura de datos

In [21]:
G = nx.from_pandas_edgelist(
    edges_df, source="source", target="target", edge_attr="type", create_using=nx.DiGraph()
)

print("Nodos:", G.number_of_nodes())
print("Aristas:", G.number_of_edges())

edges_df.to_csv("red_interacciones.csv", index=False)

Nodos: 2720
Aristas: 7336


El grafo de interacciones creado contiene los nodos representan los 2720 usuarios únicos que aparecen en las conversaciones, mientras que las aristas son 7336 relaciones (menciones, retweets, respuestas). 

Con este grafo ya se puede realizar el análisis de redes sociales.

